# 网上找了个层次分类的数据

处理一下, 两级的, 数据也不多, 大概 7 万条. 坑爹, 这个原始文件不太行

In [2]:
import os
import json
import re
from collections import defaultdict
import pandas as pd
from sklearn.model_selection import train_test_split

In [21]:
# 原始数据集有点坑, 用 excel 重新打开过, 然后删了几列, 不然有些列的长度不同
raw_file = r"G:\dataset\text_classify\网页层次分类\raw\hierarchical_data.csv"
output_dir = r"G:\dataset\text_classify\网页层次分类"

df = pd.read_csv(raw_file, header=0, low_memory=False)
print(df.shape)
df.head(3)

(65605, 8)


,id,first_level,second_level,domain,title,description,keywords,flag
0,1,休闲娱乐,影视音乐,http://www.q6636.com,奇乐电影网,奇乐电影网包含电影、电视剧、综艺、动漫、纪录片、电视直播、最新大片、最新大片电影、等热门栏目...,NaN,3
1,2,休闲娱乐,影视音乐,http://video.baidu.com,百度视频搜索,百度视频搜索是业界领先的中文视频搜索引擎之一，拥有海量的中文视频资源，提供用户满意的观看体验...,"热播电视剧,高清电影,好看的电视剧,好看的电影,电影天堂,电视剧排行榜,电影排行榜,百度影音...",2
2,3,休闲娱乐,影视音乐,http://www.dj881.com,中国DJ前线,"dj下载网为您提供破解绿色免费软件下载基地,免费绿色软件下载,共享软件基地,破解绿色软件免费下载","dj下载,免费下载,绿色下载,安全下载",1


In [24]:
is_id = re.compile(r"^\d+$")
filer = df["id"].apply(lambda x:bool(is_id.match(str(x))))
df = df[filer]
print(df.shape)

(65581, 8)


In [25]:
label0_name = "first_level"
label1_name = "second_level"
# 先构建标签
label_dict = defaultdict(int)
for i, row in df.iterrows():
    label0 = row[label0_name]
    label1 = row[label1_name]
    if pd.isna(label0) or pd.isna(label1):
        continue
    label_dict[label0] += 1
    label_dict[f"{label0}>{label1}"] += 1

print(len(label_dict))

48


In [26]:
label_dict

defaultdict(int,
            {'休闲娱乐': 4564,
             '休闲娱乐>影视音乐': 1290,
             '休闲娱乐>游戏动漫': 739,
             '休闲娱乐>图片小说': 746,
             '休闲娱乐>聊天交友': 552,
             '休闲娱乐>娱乐其他': 798,
             '休闲娱乐>休闲健身': 439,
             '电脑网络': 8915,
             '电脑网络>商务门户': 1673,
             '电脑网络>网站资源': 3885,
             '电脑网络>软硬件通信': 2010,
             '电脑网络>网络其他': 632,
             '电脑网络>网络营销': 715,
             '商业经济': 22068,
             '商业经济>农林牧渔': 999,
             '商业经济>工业制品': 5044,
             '商业经济>机械电子': 6360,
             '商业经济>建筑环境': 1641,
             '商业经济>法律金融': 1031,
             '商业经济>商务服务': 6523,
             '商业经济>交通物流': 470,
             '生活服务': 17253,
             '生活服务>生活用品': 2444,
             '生活服务>餐饮美食': 942,
             '生活服务>房产家居': 2438,
             '生活服务>旅游交通': 2412,
             '生活服务>百货购物': 1771,
             '生活服务>医疗保健': 2888,
             '生活服务>时尚美容': 1202,
             '生活服务>生活常识': 832,
             '生活服务>生活其他': 2324,
             '教育文化'

In [28]:
label_file = os.path.join(output_dir, "labels.json")
with open(label_file, "w", encoding="utf-8") as f:
    new_label_dict = {k:i for i, k in enumerate(label_dict.keys())}
    json.dump(new_label_dict, f, ensure_ascii=False, indent=2)

In [46]:
X = []
Y = []
for i, row in df.iterrows():
    label0 = row[label0_name]
    label1 = row[label1_name]
    if pd.isna(label0) or pd.isna(label1):
        continue

    title = row["title"]
    description = row["description"]
    # assert pd.notna(title) or pd.notna(description)
    title = "" if pd.isna(title) else title
    description = "" if pd.isna(description) else description
    content = f"{title} {description}".strip()
    # 移除空格和换行符等
    content = re.sub(r"\s+", " ", content)
    # TODO: 很多是乱码, 应该要移除
    if re.compile(r"[\u4e00-\u9fa5]").search(content) is None:
        continue
    if not content:
        continue

    label0_id = new_label_dict[label0]
    label1_id = new_label_dict[f"{label0}>{label1}"]

    X.append(content)
    Y.append((label0_id, label1_id))

print(len(X), len(Y))

65131 65131


In [47]:
X[:5], Y[:5]

(['奇乐电影网 奇乐电影网包含电影、电视剧、综艺、动漫、纪录片、电视直播、最新大片、最新大片电影、等热门栏目。影片使用QVOD、优酷等播放器、影片清晰流畅、网站界面清新友好、无弹窗广告的绿色电影网。每天有专人准时更新影片、是',
  '百度视频搜索 百度视频搜索是业界领先的中文视频搜索引擎之一，拥有海量的中文视频资源，提供用户满意的观看体验。在百度视频，您可以便捷地找到海量的互联网视频，更有丰富的视频榜单、多样的视频专题满足您不同的视频观看需求。百度视频，你的视界。',
  '中国DJ前线 dj下载网为您提供破解绿色免费软件下载基地,免费绿色软件下载,共享软件基地,破解绿色软件免费下载',
  'BBB119免费电影 提供免费电影、最新电影、在线电影、电影下载',
  "56我乐网 56网是中国原创视频网站,免费上传搞笑逗趣生活视频，观看优质丰富的特色节目，关注感兴趣的原创导演和美女解说，快速分享及评论互动。'"],
 [(0, 1), (0, 1), (0, 1), (0, 1), (0, 1)])

In [48]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.2, random_state=42)

train_file = os.path.join(output_dir, "train.csv")
with open(train_file, "w", encoding="utf-8") as f:
    for x, y in zip(train_X, train_Y):
        f.write(f"{x}\t{y[0]}\t{y[1]}\n")

dev_file = os.path.join(output_dir, "dev.csv")
with open(dev_file, "w", encoding="utf-8") as f:
    for x, y in zip(test_X, test_Y):
        f.write(f"{x}\t{y[0]}\t{y[1]}\n")

# 试试 paddlenlp

In [5]:
train_file = r"G:\dataset\text_classify\网页层次分类\train.csv"
dev_file = r"G:\dataset\text_classify\网页层次分类\test.csv"
label_file = r"G:\dataset\text_classify\网页层次分类\label.json"

with open(label_file, "r", encoding="utf-8") as f:
    label2id = json.load(f)
    label2id = {k: str(v) for k, v in label2id.items()}
    id2label = {v: k for k, v in label2id.items()}

In [7]:
output_dir = r"G:\dataset\text_classify\网页层次分类\paddlenlp"
os.makedirs(output_dir, exist_ok=True)

# 标签文件应该使用 ## 分隔层次的
with open(os.path.join(output_dir, "label.txt"), "w", encoding="utf-8") as f:
    for label in label2id.keys():
        label = label.replace(">", "##")
        f.write(f"{label}\n")

# 训练文件中应该使用 \t 分隔, 标签用 , 分隔
with open(train_file, "r", encoding="utf-8") as fr:
    with open(os.path.join(output_dir, "train.txt"), "w", encoding="utf-8") as fw:
        for line in fr:
            line = line.strip()
            if not line:
                continue
            line = line.split("\t")
            label0 = id2label[line[1]].replace(">", "##")
            label1 = id2label[line[2]].replace(">", "##")
            fw.write(f"{line[0]}\t{label0},{label1}\n")

# 测试文件中应该使用 \t 分隔, 标签用 , 分隔
with open(dev_file, "r", encoding="utf-8") as fr:
    with open(os.path.join(output_dir, "test.txt"), "w", encoding="utf-8") as fw:
        for line in fr:
            line = line.strip()
            if not line:
                continue
            line = line.split("\t")
            label0 = id2label[line[1]].replace(">", "##")
            label1 = id2label[line[2]].replace(">", "##")
            fw.write(f"{line[0]}\t{label0},{label1}\n")

# 再换个数据集, 使用百度的事件抽取数据集 baidu_extract_2020

这个数据是多标签的, 即是多层级, 也是多标签的

In [11]:
train_file = r"G:\dataset\text_classify\baidu_extract_2020\raw\train.txt"
dev_file = r"G:\dataset\text_classify\baidu_extract_2020\raw\dev.txt"
label_file = r"G:\dataset\text_classify\baidu_extract_2020\raw\label.txt"

label_dict = dict()
with open(label_file, "r", encoding="utf-8") as f:
    index = 0
    for line in f:
        label = line.strip().replace("##", ">")
        label_dict[label] = index
        index += 1

In [15]:
output_dir = r"G:\dataset\text_classify\baidu_extract_2020\fewshot"
os.makedirs(output_dir, exist_ok=True)

# 保存标签
with open(os.path.join(output_dir, "label.json"), "w", encoding="utf-8") as f:
    json.dump(label_dict, f, ensure_ascii=False, indent=2)

# 保存训练数据
with open(train_file, "r", encoding="utf-8") as fr:
    with open(os.path.join(output_dir, "train.csv"), "w", encoding="utf-8") as fw:
        for line in fr:
            line = line.strip()
            if not line:
                continue
            line = line.split("\t")
            label_list = line[1].replace("##", ">")
            label_id_list = [label_dict[x] for x in label_list.split(",")]
            label_id_list = "\t".join(map(str, label_id_list))
            fw.write(f"{line[0]}\t{label_id_list}\n")

# 保存测试数据
with open(dev_file, "r", encoding="utf-8") as fr:
    with open(os.path.join(output_dir, "dev.csv"), "w", encoding="utf-8") as fw:
        for line in fr:
            line = line.strip()
            if not line:
                continue
            line = line.split("\t")
            label_list = line[1].replace("##", ">")
            label_id_list = [label_dict[x] for x in label_list.split(",")]
            label_id_list = "\t".join(map(str, label_id_list))
            fw.write(f"{line[0]}\t{label_id_list}\n")

# 继续换数据集, 这次是百度的事件数据集, 但是小样本版本的

In [20]:
train_file = r"G:\dataset\text_classify\baidu_events\raw\train.txt"
dev_file = r"G:\dataset\text_classify\baidu_events\raw\dev.txt"
label_file = r"G:\dataset\text_classify\baidu_events\raw\label.txt"

label_dict = dict()
with open(label_file, "r", encoding="utf-8") as f:
    index = 0
    for line in f:
        # == 后面的是提示标签, 就先不用了
        label = line.strip().split("==")[0].replace("##", ">")
        label_dict[label] = index
        index += 1

In [22]:
output_dir = r"G:\dataset\text_classify\baidu_events\fewshot"
os.makedirs(output_dir, exist_ok=True)

# 保存标签
with open(os.path.join(output_dir, "label.json"), "w", encoding="utf-8") as f:
    json.dump(label_dict, f, ensure_ascii=False, indent=2)

# 保存训练数据
with open(train_file, "r", encoding="utf-8") as fr:
    with open(os.path.join(output_dir, "train.csv"), "w", encoding="utf-8") as fw:
        for line in fr:
            line = line.strip()
            if not line:
                continue
            line = line.split("\t")
            label_list = line[1].replace("##", ">")
            label_id_list = [label_dict[x] for x in label_list.split(",")]
            label_id_list = "\t".join(map(str, label_id_list))
            fw.write(f"{line[0]}\t{label_id_list}\n")

# 保存测试数据
with open(dev_file, "r", encoding="utf-8") as fr:
    with open(os.path.join(output_dir, "dev.csv"), "w", encoding="utf-8") as fw:
        for line in fr:
            line = line.strip()
            if not line:
                continue
            line = line.split("\t")
            label_list = line[1].replace("##", ">")
            label_id_list = [label_dict[x] for x in label_list.split(",")]
            label_id_list = "\t".join(map(str, label_id_list))
            fw.write(f"{line[0]}\t{label_id_list}\n")